#### Import Libraries

In [97]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from pyspark.sql import functions as F  #filtering
import pandas as pd
import geopandas as gpd
import folium

### Inspect data

In [98]:
# Starting a Spark session
spark = (
    SparkSession.builder.appName('Metropolitan Victoria Findings')
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "2g")
    .config("spark.executor.memory", "4g")
    .getOrCreate()
)

In [99]:
# Read the shapefile using GeoPandas
vic_boundaries = gpd.read_file('../data/landing/boundaries/Victoria/vic_dist_boundaries.shp')

vic_boundaries.head(1)

,sa2_code,sa2_name,chg_flag,chg_lbl,sa3_code,sa3_name,sa4_code,sa4_name,gcc_code,gcc_name,ste_code,ste_name,aus_code,aus_name,areasqkm,loci_uri,geometry
0,201011001,Alfredton,0,No change,20101,Ballarat,201,Ballarat,2RVIC,Rest of Vic.,2,Victoria,AUS,Australia,52.7109,http://linked.data.gov.au/dataset/asgsed3/SA2/...,"POLYGON ((143.78281 -37.56667, 143.75557 -37.5..."


In [100]:
print(len(vic_boundaries))

522


In [101]:
# Read suburb parquet data into a Spark DataFrame
suburb_spark_df = spark.read.parquet('../data/landing/suburb_match/suburb_match.parquet')

# Filter the Spark DataFrame to only include rows where the state is 'VIC'
suburb_spark_df = suburb_spark_df.filter(suburb_spark_df['state'] == 'VIC')

# Rename the 'SA2_NAME_2021' column to 'sa2_name'
suburb_spark_df = suburb_spark_df.withColumnRenamed('SA2_NAME_2021', 'sa2_name')

# Display the first row
suburb_spark_df.limit(1).show()

+----+--------+---------+-----+-----------+------------+--------------------+-------------+--------------------+-------+--------------+-----+-----------------+------+-----------+------------+---------------+---------------+-------------+---------------+-------------+--------------------+-------------+--------------------+-------+-------+-------+--------------------+--------+--------+---------+----------------+----------+--------+--------------------+---------+-------+----------+------------------+--------+--------------------+
|  id|postcode| locality|state|       long|         lat|                  dc|         type|              status|    sa3|       sa3name|  sa4|          sa4name|region|Lat_precise|Long_precise|  SA1_CODE_2021|  SA1_NAME_2021|SA2_CODE_2021|       sa2_name|SA3_CODE_2021|       SA3_NAME_2021|SA4_CODE_2021|       SA4_NAME_2021|RA_2011|RA_2016|RA_2021|        RA_2021_NAME|MMM_2015|MMM_2019|      ced|        altitude|chargezone|phn_code|            phn_name|lgaregion|lga

In [102]:
# Convert suburb_spark_df to Pandas DataFrame (if needed)
suburb_df = suburb_spark_df.toPandas()

print(len(suburb_df))

3541


### Find the corresponding postcode for each SA2 district

In [103]:
# Create a DataFrame with unique sa2_name and their corresponding first postcode
unique_postcodes = suburb_df[['sa2_name', 'postcode']].drop_duplicates().groupby('sa2_name', as_index=False).first()

# Merge again with unique postcodes
merged_data = vic_boundaries.merge(unique_postcodes, on='sa2_name', how='left')

merged_data[['sa2_name', 'postcode']].head()

,sa2_name,postcode
0,Alfredton,NaN
1,Ballarat,NaN
2,Buninyong,3357.0
3,Delacombe,3358.0
4,Smythes Creek,NaN


In [104]:
# Identify rows with NaN postcodes
nan_postcodes = merged_data[merged_data['postcode'].isna()]
print(len(nan_postcodes))

234


In [105]:
# Iterate through NaN entries and attempt to find postcode based on locality
for index, row in nan_postcodes.iterrows():
    locality_name = ""

    # Check if the sa2_name contains a hyphen
    if '-' in row['sa2_name']:
        # Extract the part after the hyphen
        locality_name = row['sa2_name'].split('-')[0].strip()  # Get the part before hyphen
    else:
        # Handle cases with parentheses
        locality_name = row['sa2_name'].split('(')[0].strip()  # Get the part before any parentheses

    # Check if the locality in suburb_df matches the locality name
    matching_rows = suburb_df[suburb_df['locality'].str.lower() == locality_name.lower()]

    if not matching_rows.empty:
        # Fill postcode from the first matching row
        merged_data.at[index, 'postcode'] = matching_rows.iloc[0]['postcode']
    else:
        # If no match found, try using the full sa2_name as a fallback
        fallback_matching_rows = suburb_df[suburb_df['locality'].str.lower() == row['sa2_name'].lower()]
        if not fallback_matching_rows.empty:
            merged_data.at[index, 'postcode'] = fallback_matching_rows.iloc[0]['postcode']

# Display the updated merged DataFrame to check results
print(merged_data[['sa2_name', 'postcode']].head(50))

                         sa2_name  postcode
0                       Alfredton    3350.0
1                        Ballarat    3350.0
2                       Buninyong    3357.0
3                       Delacombe    3358.0
4                   Smythes Creek    3351.0
5         Wendouree - Miners Rest    3355.0
6      Ballarat East - Warrenheip    3350.0
7       Ballarat North - Invermay    3350.0
8          Canadian - Mount Clear    3350.0
9              Sebastopol - Redan    3350.0
10        Bacchus Marsh Surrounds       NaN
11              Creswick - Clunes    3363.0
12                     Daylesford    3363.0
13                  Gordon (Vic.)    3334.0
14                          Avoca    3467.0
15                       Beaufort    3468.0
16          Golden Plains - North    3360.0
17             Maryborough (Vic.)    3465.0
18          Maryborough Surrounds    3370.0
19                        Bendigo    3550.0
20   California Gully - Eaglehawk    3556.0
21      East Bendigo - Kenningto

In [106]:
# Identify rows with NaN postcodes
nan_postcodes = merged_data[merged_data['postcode'].isna()]
print(len(nan_postcodes))

17


In [107]:
nan_postcodes['sa2_name']

10                           Bacchus Marsh Surrounds
27                         Bendigo Surrounds - South
125                     Essendon (West) - Aberfeldie
130                            Flemington Racecourse
135                             Melbourne CBD - East
136                            Melbourne CBD - North
137                             Melbourne CBD - West
139                   Royal Botanic Gardens Victoria
140                   Southbank (West) - South Wharf
162                      Richmond (South) - Cremorne
192                      Highett (West) - Cheltenham
250                             Epping (Vic.) - West
261                                 Essendon Airport
308    Ferntree Gully (South) - Upper Ferntree Gully
392                   Clayton (North) - Notting Hill
446                  Tarneit (West) - Mount Cottrell
475                                Horsham Surrounds
Name: sa2_name, dtype: object

In [108]:
# Define a dictionary for manual postcode updates
manual_postcode_updates = {
    'Bacchus Marsh Surrounds': '3340',  
    'Bendigo Surrounds - South': '3550',
    'Essendon (West) - Aberfeldie': '3042',
    'Flemington Racecourse': '3031',
    'Melbourne CBD - East': '3000',
    'Melbourne CBD - North': '3000',
    'Melbourne CBD - West': '3000',
    'Royal Botanic Gardens Victoria': '3004',
    'Southbank (West) - South Wharf': '3006',
    'Richmond (South) - Cremorne': '3121',
    'Highett (West) - Cheltenham': '3190',
    'Epping (Vic.) - West': '3076',
    'Essendon Airport': '3041',
    'Ferntree Gully (South) - Upper Ferntree Gully': '3156',
    'Clayton (North) - Notting Hill': '3168',
    'Tarneit (West) - Mount Cottrell': '3029',
    'Horsham Surrounds': '3400'
}

# Update the postcode for rows with NaN values using the manual updates
for sa2_name, postcode in manual_postcode_updates.items():
    merged_data.loc[merged_data['sa2_name'].str.lower() == sa2_name.lower(), 'postcode'] = postcode

# Convert postcode column to string and remove decimal points (if any)
merged_data['postcode'] = merged_data['postcode'].astype(str).str.replace('.0', '', regex=False)

# Display the updated merged DataFrame to check results
print(merged_data[['sa2_name', 'postcode']].head(20))

                      sa2_name postcode
0                    Alfredton     3350
1                     Ballarat     3350
2                    Buninyong     3357
3                    Delacombe     3358
4                Smythes Creek     3351
5      Wendouree - Miners Rest     3355
6   Ballarat East - Warrenheip     3350
7    Ballarat North - Invermay     3350
8       Canadian - Mount Clear     3350
9           Sebastopol - Redan     3350
10     Bacchus Marsh Surrounds     3340
11           Creswick - Clunes     3363
12                  Daylesford     3363
13               Gordon (Vic.)     3334
14                       Avoca     3467
15                    Beaufort     3468
16       Golden Plains - North     3360
17          Maryborough (Vic.)     3465
18       Maryborough Surrounds     3370
19                     Bendigo     3550


/tmp/ipykernel_1715/2832867521.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3340' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merged_data.loc[merged_data['sa2_name'].str.lower() == sa2_name.lower(), 'postcode'] = postcode


In [109]:
# Identify rows with NaN postcodes
nan_postcodes = merged_data[merged_data['postcode'].isna()]
print(len(nan_postcodes))

0


In [110]:
# Filter the merged_data for postcodes between 3000 and 3200
filtered_data = merged_data[(merged_data['postcode'].astype(int) >= 3000) & (merged_data['postcode'].astype(int) <= 3200)]

# Select only the sa2_name column
filtered_data = filtered_data[['sa2_name']]

# Manually add new sa2_name values since they are metropolitan but have a postcode out of the range
new_sa2_names = pd.DataFrame({'sa2_name': ['Whittlesea', 'Werribee - West', 'Wandin - Seville', 'Mernda - North', 'Doreen - North',
                                           'Doreen - South', 'Mernda - South', 'South Morang - North', 'South Morang - South', 'Wollert',
                                           'Wyndham Vale - North', 'Mount Evelyn', 'Montrose']})

# Append the new sa2_name entries to the filtered data
filtered_data = pd.concat([filtered_data, new_sa2_names], ignore_index=True)

# Display the result
filtered_data.head()

,sa2_name
0,Brunswick East
1,Brunswick West
2,Pascoe Vale South
3,Brunswick - North
4,Brunswick - South


In [111]:
len(filtered_data)

292

### Save

In [112]:
# Save the filtered DataFrame to a CSV file
output_file_path = '../data/curated/metropolitan_victoria.csv'
filtered_data.to_csv(output_file_path, index=False)